### Libraries and local code imports

In [2]:
import numpy as np
from numpy import nan

np.random.seed(10)
import math
from enum import Enum
import networkx as nx
import matplotlib.pyplot as plt
from itertools import product
import random
import pandas as pd
#necessary for removing dublicates from a list of lists
import itertools
import datetime
#parallel processing
#import pathos
from pathos.multiprocessing import ProcessPool
#from multiprocessing import Pool                                                
import ast
import six


#mesa imports

from mesa import Agent, Model
from mesa.time import StagedActivation
from mesa.datacollection import DataCollector
from mesa.space import NetworkGrid
from mesa.batchrunner import BatchRunner,BatchRunnerMP


In [57]:
def create_mean_cols(m_data,model_no):
    #read data
    susdata=m_data.Susceptible.to_frame().reset_index()
    infdata=m_data.Infected.to_frame().reset_index()
    immdata=m_data.Immune.to_frame().reset_index()
    totalinfdata=m_data.TotalInfected.to_frame().reset_index()
    reffdata=m_data.Reff.to_frame().reset_index()
    reffATdata=m_data.ReffAT.to_frame().reset_index()
    meanSus=[]
    meanInf=[]
    meanImm=[]
    meanInfperPerson=[]
    meanInfperInfected=[]
    meanReff=[]
    meanReffAT=[]
    for i in range(len(susdata.Susceptible)): 
        meanSus.append(np.average(susdata.Susceptible[i]))
        meanInf.append(np.average(infdata.Infected[i])) 
        meanImm.append(np.average(immdata.Immune[i])) 
        meanInfperPerson.append(np.sum(infdata.Infected[i])/(m_data.TotalPop[0]))
        meanInfperInfected.append(np.sum(infdata.Infected[i])/(totalinfdata.TotalInfected[i]))
        meanReff.append(np.nanmean(reffdata.Reff[i]))
        meanReffAT.append(np.nanmean(reffATdata.ReffAT[i]))
        
        #meanSus.append(np.average(ast.literal_eval(susdata.Susceptible[i]))) 
        #meanInf.append(np.average(ast.literal_eval(infdata.Infected[i]))) 
        #meanImm.append(np.average(ast.literal_eval(immdata.Immune[i])))
    meanImm2=[a_i - b_i for a_i, b_i in zip(meanImm, meanInf)]
    #add new column to pd df
    #https://www.geeksforgeeks.org/adding-new-column-to-existing-dataframe-in-pandas/
    m_data['meanSusceptible'] = meanSus 
    m_data['meanImmune'] = meanImm2
    m_data['meanInfected'] = meanInf
    m_data['meanInfectionperPerson'] = meanInfperPerson
    m_data['meanInfectionperInfected'] = meanInfperInfected
    m_data['meanReff']=meanReff
    m_data['meanReffAT']=meanReffAT
    #add model col
    m_data['ModelNo']=model_no
    return m_data



In [ ]:
#https://stackoverflow.com/questions/26678467/export-a-pandas-dataframe-as-a-table-image
def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')

    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)

    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in  six.iteritems(mpl_table._cells):
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return fig
